CSCI-513 Project\
April 2025\
Lon Cherryhomes, Sr.

In [1]:
import time
from pprint import pprint, pformat

In [2]:
!pip install Cython
%load_ext cython

In [3]:
%%cython

cdef int pos
cdef bytes subject

cdef void factor():
    global pos, subject
    if   subject[pos] == ord('+'): pos += 1; factor()
    elif subject[pos] == ord('-'): pos += 1; factor()
    elif subject[pos] == ord('('):
        pos += 1
        expr()
        if subject[pos] == ord(')'): pos += 1
        else: raise Exception(pos)
    elif subject[pos] == ord('x'): pos += 1
    elif subject[pos] == ord('y'): pos += 1
    elif subject[pos] == ord('0'): pos += 1
    elif subject[pos] == ord('1'): pos += 1
    else: raise Exception(pos)

cdef void term():
    global pos, subject
    factor()
    while (  subject[pos] == ord('*')
          or subject[pos] == ord('/')
          ):
          pos += 1
          term()

cdef void expr():
    global pos, subject
    term()
    while (  subject[pos] == ord('+')
          or subject[pos] == ord('-')
          ):
          pos += 1
          expr()

cdef void stmt():
    global pos, subject
    expr()
    if subject[pos] != ord('\n'):
        raise Exception(pos)

def parse_expression(s):
    global pos; pos = 0
    global subject; subject = f"{s}\n".encode('ascii')
    try: stmt()
    except Exception as e:
        return e.args[0]+1
    return 0

In [4]:
from itertools import product
total = None
tokens = "(x*-1+0/y)"
def expressions(length):
    global total; total = 1
    for toks in product(tokens, repeat=length):
        expression = "".join(toks)
        yield (parse_expression(expression), expression)
        total += 1
    total -= 1

In [5]:
for length in range(1, 8):
    start = time.time_ns() // 1000
    classes = [0] * (length+2)
    for expr in expressions(length):
        classes[expr[0]] += 1
    finish = time.time_ns() // 1000
    classes = [(c * 100.0 / total) for c in classes]
    pprint([length, total, (finish - start) / 1000000.0, classes])

[1, 10, 0.006108, [40.0, 30.0, 30.0]]
[2, 100, 0.000289, [8.0, 30.0, 33.0, 29.0]]
[3, 1000, 0.007784, [8.4, 30.0, 33.0, 14.3, 14.3]]
[4, 10000, 0.096411, [3.04, 30.0, 33.0, 14.3, 9.73, 9.93]]
[5, 100000, 0.484053, [2.1, 30.0, 33.0, 14.3, 9.73, 5.219, 5.651]]
[6, 1000000, 4.564928, [0.9624, 30.0, 33.0, 14.3, 9.73, 5.219, 3.2049, 3.5837]]
[7,
 10000000,
 27.392036,
 [0.57892, 30.0, 33.0, 14.3, 9.73, 5.219, 3.2049, 1.83727, 2.12991]]


In [6]:
token_map = {'(': 1, 'x': 2, '0': 3, '+': 4, '*': 5, '/': 6, '-': 7, '1': 8, 'y': 9, ')': 10}
def EXPRESSIONS(length):
    XS = []
    for expr in expressions(length):
        XS.append([expr[0]] + [token_map[x] for x in expr[1]])
    return XS

In [7]:
import numpy as np
import pandas as pd
import warnings
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
#-------------------------------------------------------------------------------
results = {}
warnings.filterwarnings("ignore")
for length in range(1, 5):
    columns = ['target'] + [f"x{i}" for i in range(1, length+1)]
    data = pd.DataFrame(EXPRESSIONS(length), columns=columns)
    X = data.iloc[:, 1:length+1]
    y = data.iloc[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=53, stratify=y
    )
    models = {
        "Logistic Regression": LogisticRegression(max_iter=200),
        "MultinomialNB": MultinomialNB(),
        "Support Vector Classifier": SVC(),
        "Random Forest Classifier": RandomForestClassifier(n_estimators=100),
        "Decision Tree Classifier": DecisionTreeClassifier()
    }
#   ----------------------------------------------------------------------------
    results[length] = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[length][model_name] = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, average='weighted'),
            "recall": recall_score(y_test, y_pred, average='weighted'),
            "F1": f1_score(y_test, y_pred, average='weighted')
        }
        print(f"sklearn: {length}. {model_name}")
        print(classification_report(y_test, y_pred))
#   ----------------------------------------------------------------------------
    comparison = pd.DataFrame(
    { model_name: {
          "accuracy": metrics["accuracy"],
          "precision": metrics["precision"],
          "recall": metrics["recall"],
          "F1": metrics["F1"]
      } for model_name, metrics in results[length].items()
    }).T
    print(f"sklearn: {length}. Comparison")
    print(comparison)
    print()
#-------------------------------------------------------------------------------

sklearn: 1. Logistic Regression
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0

sklearn: 1. MultinomialNB
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3

sklearn: 1. Support Vector Classifier
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00

In [8]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [9]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
#-------------------------------------------------------------------------------
vocab_size = 11
embedding_dim = 8
hidden_dim = 32
LEARNING_RATE = 0.001
EPOCHS = 20
#-------------------------------------------------------------------------------
# Model 1: Logistic Regression (a single linear layer after embedding+flattening)
class LogisticRegressionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, seq_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(seq_length * embedding_dim, num_classes)

    def forward(self, x):
        # x: [batch_size, seq_length]
        x = self.embedding(x) # shape: [batch_size, seq_length, embedding_dim]
        x = x.view(x.size(0), -1) # flatten to [batch_size, seq_length*embedding_dim]
        out = self.fc(x)
        return out
#-------------------------------------------------------------------------------
# Model 2: Feedforward network with one hidden layer
class FeedforwardNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(seq_length * embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        return out
#-------------------------------------------------------------------------------
# Model 3: Deeper network with two hidden layers
class DeeperNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(seq_length * embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        out = self.fc3(x)
        return out
#-------------------------------------------------------------------------------
# Model 4: RNN using an LSTM (the hidden state of the last time step is used for classification)
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, hidden_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x) # [batch, seq_length, embedding_dim]
        out, (hn, cn) = self.lstm(x) # hn: [num_layers, batch, hidden_dim]
        out = self.fc(hn[-1]) # use last layer’s hidden state for classification
        return out
#-------------------------------------------------------------------------------
# Model 5: Transformer-based Model (using a single encoder layer)
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, nhead=2, num_encoder_layers=1, dim_feedforward=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Learnable positional encoding (for simplicity)
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_length, embedding_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.fc = nn.Linear(seq_length * embedding_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x) + self.pos_embedding # [batch, seq_length, embedding_dim]
        x = x.transpose(0, 1) # Transformer expects: [seq_length, batch, embedding_dim]
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1) # back to [batch, seq_length, embedding_dim]
        x = x.reshape(x.size(0), -1) # flatten
        out = self.fc(x)
        return out
#-------------------------------------------------------------------------------
# Model 6: Convolutional Neural Network (CNN) with 1D Convolutions
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, num_filters=16, kernel_size=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # For conv1d, we expect input of shape (batch, channels, seq_length)
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size)
        conv_output_size = seq_length - kernel_size + 1
        self.fc = nn.Linear(num_filters * conv_output_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.embedding(x) # [batch, seq_length, embedding_dim]
        x = x.transpose(1, 2) # [batch, embedding_dim, seq_length]
        x = self.conv1(x) # [batch, num_filters, new_seq_length]
        x = self.relu(x)
        x = x.view(x.size(0), -1) # flatten
        out = self.fc(x)
        return out
#-------------------------------------------------------------------------------
def train_model(model, train_loader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
#-------------------------------------------------------------------------------
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu())
            all_labels.append(batch_y)
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    return all_preds, all_labels
#-------------------------------------------------------------------------------
results = {}
for length in range(2, 5):
    data = np.array(EXPRESSIONS(length))
    X = data[:, 1:].astype(np.int64) # tokens (shape: [n_samples, n_columns])
    y = data[:, 0].astype(np.int64)
    num_classes = int(y.max()) + 1
    seq_length = X.shape[1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)
    batch_size = 32
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))
    test_dataset = TensorDataset(
        torch.tensor(X_test, dtype=torch.long),
        torch.tensor(y_test, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#   ----------------------------------------------------------------------------
    models = {
        "Logistic Regression": LogisticRegressionModel(vocab_size, embedding_dim, num_classes, seq_length),
        "Feedforward NN": FeedforwardNN(vocab_size, embedding_dim, num_classes, seq_length, hidden_dim),
        "Deeper NN": DeeperNN(vocab_size, embedding_dim, num_classes, seq_length, hidden_dim),
        "RNN (LSTM)": RNNModel(vocab_size, embedding_dim, num_classes, hidden_dim),
        "Transformer": TransformerModel(vocab_size, embedding_dim, num_classes, seq_length),
        "CNN": CNNModel(vocab_size, embedding_dim, num_classes, seq_length)
    }
#   ----------------------------------------------------------------------------
    results[length] = {}
    for model_name, model in models.items():
        print(f"Training model: {model_name}")
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        train_model(model, train_loader, criterion, optimizer, device, epochs=EPOCHS)
        preds, labels = evaluate_model(model, test_loader, device)
        results[length][model_name] = {
            "accuracy": accuracy_score(labels, preds),
            "precision": precision_score(labels, preds, average='weighted', zero_division=0),
            "recall": recall_score(labels, preds, average='weighted', zero_division=0),
            "F1": f1_score(labels, preds, average='weighted', zero_division=0),
        }
        print(f"PyTorch: {length}. {model_name}:")
        print(classification_report(labels, preds, zero_division=0))
#   ----------------------------------------------------------------------------
    comparison = pd.DataFrame(
    { model_name: {
          "accuracy": metrics["accuracy"],
          "precision": metrics["precision"],
          "recall": metrics["recall"],
          "F1": metrics["F1"]
      } for model_name, metrics in results[length].items()
    }).T
    print(f"PyTorch: {length}. Comparison")
    print(comparison)
    print()
#-------------------------------------------------------------------------------

Using device: cuda
Training model: Logistic Regression
Epoch [1/20], Loss: 1.7325
Epoch [2/20], Loss: 1.7108
Epoch [3/20], Loss: 1.6381
Epoch [4/20], Loss: 1.6382
Epoch [5/20], Loss: 1.6184
Epoch [6/20], Loss: 1.6016
Epoch [7/20], Loss: 1.5869
Epoch [8/20], Loss: 1.6098
Epoch [9/20], Loss: 1.5680
Epoch [10/20], Loss: 1.5762
Epoch [11/20], Loss: 1.5250
Epoch [12/20], Loss: 1.5267
Epoch [13/20], Loss: 1.5270
Epoch [14/20], Loss: 1.4872
Epoch [15/20], Loss: 1.5160
Epoch [16/20], Loss: 1.4903
Epoch [17/20], Loss: 1.4695
Epoch [18/20], Loss: 1.4696
Epoch [19/20], Loss: 1.4472
Epoch [20/20], Loss: 1.4420
PyTorch: 2. Logistic Regression:
              precision    recall  f1-score   support

           0       0.33      0.50      0.40         2
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         8
           3       0.17      0.50      0.25         4

    accuracy                           0.15        20
   macro avg       0.12      0.25  